In [2]:
!pip install numpy
!pip install time
!pip install datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 36.0 MB/s eta 0:00:0000:010:01
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [datetime]2/4 [zope.interface]


In [8]:
import numpy as np
import time
from datetime import datetime, timedelta
import random
import csv
import os

import time
import random
import numpy as np
import csv
from datetime import datetime, timedelta

# Set the total runtime to 110 minutes
total_runtime = timedelta(minutes=110)
start_time = datetime.now()

# Calculate the time after which anomalies are allowed
anomaly_free_duration = timedelta(minutes=30)
anomaly_start_time = start_time + anomaly_free_duration

print(f"Script started at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Anomalies will be prevented until {anomaly_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total runtime set to {total_runtime}")

# List of matrix sizes
normal_sizes = [1200, 1600, 1800, 2200]
anomaly_sizes = [5000]

# Create or open the CSV file for logging
log_file = 'execution_log.csv'
# Use 'w' to create/overwrite the file and write header
with open(log_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp', 'state', 'matrix_size_used']) # Added size for clarity

print(f"Logging to {log_file}")

# Main loop
while datetime.now() - start_time < total_runtime:
    current_time = datetime.now()
    time_elapsed = current_time - start_time

    # --- Anomaly Control Logic ---
    # Check if we are still within the anomaly-free period
    if current_time < anomaly_start_time:
        # Force normal behavior
        is_anomaly = False
        matrix_size = random.choice(normal_sizes) # Always pick from normal sizes
        current_state = 'normal'
        # print(f"DEBUG: {current_time.strftime('%H:%M:%S')} - Within initial {anomaly_free_duration.total_seconds()}s, forced normal (size: {matrix_size})") # Optional debug print
    else:
        # After the initial period, revert to original random logic
        # Randomly select a number between 1 and 50
        random_choice = random.randint(1, 50)

        # Determine if this will be a normal run or anomaly
        is_anomaly = (random_choice == 20)

        # Select matrix size based on the random choice
        if is_anomaly:
            matrix_size = random.choice(anomaly_sizes)
            current_state = 'anomaly'
            # print(f"DEBUG: {current_time.strftime('%H:%M:%S')} - After initial period, ANOMALY triggered (size: {matrix_size})") # Optional debug print
        else:
            matrix_size = random.choice(normal_sizes)
            current_state = 'normal'
            # print(f"DEBUG: {current_time.strftime('%H:%M:%S')} - After initial period, normal run (size: {matrix_size})") # Optional debug print
    # --- End Anomaly Control Logic ---

    # Log the start of the operation state (normal or anomaly based on size)
    with open(log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([current_time.strftime('%Y-%m-%d %H:%M:%S'), current_state, matrix_size])

    print(f"Running {current_state} operation with size {matrix_size} at {current_time.strftime('%H:%M:%S')} ({time_elapsed} elapsed)")

    try:
        # CPU intensive operation
        matrix_a = np.random.rand(matrix_size, matrix_size)
        matrix_b = np.random.rand(matrix_size, matrix_size)

        # Perform multiple matrix operations to create CPU load
        # The number of operations might need tuning depending on how long you
        # want each cycle to take. For size 5000, one dot product is already slow.
        num_operations = 1 if matrix_size >= 4000 else 3 # Reduced operations for large matrices
        for i in range(num_operations):
            print(f"  Performing dot product {i+1}/{num_operations}...")
            result = np.dot(matrix_a, matrix_b)
            # Optional: Use result for next op or just compute it
            if i < num_operations - 1:
                 # Simple way to use the result and create data dependency
                 matrix_a = result * 1e-9 # Scale down to keep numbers manageable
                 # Or just ensure matrix_a/b are ready for next iteration
                 # For simple load generation, just computing result is enough
                 pass # Keep original matrix_a, b or re-generate if preferred


        # Clear memory - might not be strictly necessary with del, but good practice
        del result, matrix_a, matrix_b
        # Optional: Force garbage collection
        # import gc
        # gc.collect()

    except Exception as e:
        print(f"Error during matrix operation: {e}")
        # Log error state if needed
        with open(log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'error', matrix_size])


    # Rest period (logged as normal state)
    # Log the end of the busy period / start of sleep
    sleep_start_time = datetime.now()
    with open(log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        # Logging 'normal' here signifies the *transition* to a non-busy state.
        # You could also log 'sleeping' if you want to distinguish.
        writer.writerow([sleep_start_time.strftime('%Y-%m-%d %H:%M:%S'), 'sleeping', 0]) # Log 0 size for sleep
    print(f"Sleeping for 1.5 seconds at {sleep_start_time.strftime('%H:%M:%S')}")

    sleep_time = 1.5
    time.sleep(sleep_time)


print("Script completed")

Script started at 2025-05-23 17:02:49
Anomalies will be prevented until 2025-05-23 17:17:49
Total runtime set to 1:50:00
Logging to execution_log.csv
Running normal operation with size 1600 at 17:02:49 (0:00:00.000831 elapsed)
  Performing dot product 1/3...
  Performing dot product 2/3...
  Performing dot product 3/3...
Sleeping for 1.5 seconds at 17:02:50
Running normal operation with size 2200 at 17:02:51 (0:00:01.699169 elapsed)
  Performing dot product 1/3...
  Performing dot product 2/3...
  Performing dot product 3/3...
Sleeping for 1.5 seconds at 17:02:52
Running normal operation with size 2200 at 17:02:53 (0:00:03.827295 elapsed)
  Performing dot product 1/3...
  Performing dot product 2/3...
  Performing dot product 3/3...
Sleeping for 1.5 seconds at 17:02:54
Running normal operation with size 1200 at 17:02:55 (0:00:05.777404 elapsed)
  Performing dot product 1/3...
  Performing dot product 2/3...
  Performing dot product 3/3...
Sleeping for 1.5 seconds at 17:02:55
Running no

KeyboardInterrupt: 